In [2]:
## 하루마다 실행

from config import Config
from mysql import connector
import requests
import time
from datetime import date
import re
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

def run_query(query):
    conn = connector.connect(**Config) 
    with conn.cursor() as cur:
        cur.execute(query)
        rows= cur.fetchall()
    conn.commit()
    conn.close()

    return rows

def get_youtube_mv() :

    driver = webdriver.Chrome()
    today = date.today().strftime('%Y-%m-%d')

    official_mv = {'ETA' : 'https://youtu.be/jOTfBlKSQYY',
    'COOL WITH YOU' : 'https://youtu.be/zsYSSVoQnP4',
    'SUPER SHY' : 'https://youtu.be/ArmDp-zijuc'
    }

    for music, url in official_mv.items() :
        driver.get(url)
        time.sleep(1)

        driver.find_element(By.XPATH, '//*[@id="expand"]').click()
        view = driver.find_element(By.XPATH, '//*[@id="info"]/span[1]').text
        view = re.sub(r'[^0-9]', '', view)
        view

        for i in range(5) :
            driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.PAGE_DOWN)

        time.sleep(3)
        comment = driver.find_element(By.XPATH, '//*[@id="count"]/yt-formatted-string/span[2]').text
        comment = re.sub(r'[^0-9]', '', comment)

        like = driver.find_element(By.XPATH, '//*[@id="segmented-like-button"]/ytd-toggle-button-renderer/yt-button-shape/button/div[2]/span').text
        like = like.replace("천", "000")
        like = like.replace("만","0000")
        like

        run_query(f'INSERT INTO streaming_table VALUES ("{today}", "{music}", "youtube_official", {int(view)}, {int(comment)}, {int(like)})')


if __name__ == '__main__' :
    get_youtube_mv()

